a. BASIC OBJECT CLASSIFICATION USING PRE-TRAINED VGG16 MODEL

In [2]:
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.utils import to_categorical

# Load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Load the VGG16 model without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Create data generators for data augmentation
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_test, y_test), epochs=5, callbacks=[early_stopping])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727275335.010731      96 service.cc:145] XLA service 0x7cec7400f940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727275335.010793      96 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   8/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.1017 - loss: 17.1536

I0000 00:00:1727275337.483601      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 54s 32ms/step - accuracy: 0.4693 - loss: 3.2503 - val_accuracy: 0.6165 - val_loss: 1.1318
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 29ms/step - accuracy: 0.6059 - loss: 1.1279 - val_accuracy: 0.6382 - val_loss: 1.0715
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.6312 - loss: 1.0581 - val_accuracy: 0.6471 - val_loss: 1.0485
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 47s 30ms/step - accuracy: 0.6448 - loss: 1.0245 - val_accuracy: 0.6488 - val_loss: 1.0721
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.6482 - loss: 1.0028 - val_accuracy: 0.6508 - val_loss: 1.0374
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6525 - loss: 1.0309
Test Accuracy: 65.08%


# b. IMAGENET CLASSIFICATION WITH DEEP RESIDUAL NETWORKS (RESNET)

In [4]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

# Load an image file that contains an image to be classified
img_path = '/kaggle/input/catcat/train/dogs/dog_10.jpg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a batch dimension (since the model expects a batch of images)
x = np.expand_dims(x, axis=0)

# Preprocess the input image for the model
x = preprocess_input(x)

# Predict the class of the image
predictions = model.predict(x)

# Decode the top 3 predictions into human-readable class names
print('Predicted:', decode_predictions(predictions, top=3)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: [('n02105162', 'malinois', 0.804456), ('n03803284', 'muzzle', 0.050654963), ('n02106662', 'German_shepherd', 0.035840467)]


# C. CLASSIFYING SPECIES OF FLOWERS USING TRANSFER LEARNING

In [6]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Set up data directories
train_dir = '/kaggle/input/flower-classification-5-classes-roselilyetc/Flower Classification/Flower Classification/Testing Data'
valid_dir = '/kaggle/input/flower-classification-5-classes-roselilyetc/Flower Classification/Flower Classification/Training Data'

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')

# Load the pre-trained VGG16 model without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(train_generator, epochs=5, validation_data=valid_generator, callbacks=[early_stopping])

# Evaluate the model on validation data
loss, accuracy = model.evaluate(valid_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Found 958 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Epoch 1/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4145 - loss: 3.3267 - val_accuracy: 0.6896 - val_loss: 0.8898
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 480ms/step - accuracy: 0.8097 - loss: 0.5312 - val_accuracy: 0.7250 - val_loss: 0.8053
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 477ms/step - accuracy: 0.9604 - loss: 0.1881 - val_accuracy: 0.7444 - val_loss: 0.7877
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 459ms/step - accuracy: 0.9878 - loss: 0.1073 - val_accuracy: 0.7310 - val_loss: 0.8181
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 469ms/step - accuracy: 1.0000 - loss: 0.0458 - val_accuracy: 0.7536 - val_loss: 0.7745
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step - accuracy: 0.7415 - loss: 0.8198
Validation Accuracy: 75.36%
